In [13]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])
    #str.startswith(pat, na=nan)查看元素是否以…开头
    #isalpha() 方法检测字符串是否只由字母组成

In [174]:
def pat_match(pattern, saying):
    #print(pattern[0:])#为什么不能打印[0]??
    if not pattern or not saying: return []#最后pattern or saying中没有列表
    
    if is_variable(pattern[0]):return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])#如果pattern[0]以"?"开头
    else:
        if pattern[0] != saying[0]: return False#对应不相等
        else:
            return pat_match(pattern[1:], saying[1:])#递归

In [144]:
got_patterns = pat_match("I want ?X".split(), "I want iPhone".split())

['I', 'want', '?X']
['want', '?X']
['?X']
[]


In [145]:
got_patterns

[('?X', 'iPhone')]

In [221]:
def pat_to_dict(patterns):
    return {k: v for k, v in patterns}#{'?X': 'iPhone'}，patterns形成对应列表

In [222]:
def subsitite(rule, parsed_rules):
    if not rule: return []
    
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)#查找字典
    # get() 字典，函数返回指定键的值，如果值不在字典中返回默认值default。

In [223]:
pat_to_dict(got_patterns)

{'?X': 'holiday'}

In [224]:
subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns))

['What', 'if', 'you', 'mean', 'if', 'you', 'got', 'a', 'holiday']

In [225]:
' '.join(subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns)))

'What if you mean if you got a holiday'

In [226]:
#案例

In [295]:
pattern = 'I want ?X'
rules = 'What if you mean if you got a ?X'

In [296]:
def get_response(saying, rules):
    got_patterns = pat_match(pattern.split(), saying.split())
    #subsitite(rules.split(), pat_to_dict(got_patterns))
    return ' '.join(subsitite(rules.split(), pat_to_dict(got_patterns)))

In [297]:
get_response( saying = "I want apple",rules = rules)

'What if you mean if you got a apple'

In [230]:
#示例

In [231]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

In [232]:
def get_response(saying, rules):
    """" please implement the code, to get the response as followings:
    
    >>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """
    pass

In [ ]:
#升级多个词

In [1]:
from collections import defaultdict

In [2]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])
is_pattern_segment('?*P')

True

In [3]:
def is_match(rest, saying):
    #print(rest[0:])###???????
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

In [4]:
#解决多个词相互对应问题

In [5]:
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')#replace(old,new)方法把字符串中的 old（旧字符串） 替换成 new(新字符串)

    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i
    
    return (seg_pat, saying), len(saying)

In [6]:
segment_match('?*P is very good'.split(), "My dog and my cat is very good".split())

(('?P', ['My', 'dog', 'and', 'my', 'cat']), 5)

In [14]:
fail = [True, None]

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

In [15]:
def subsitite(rule, parsed_rules):
    if not rule: return []
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)#查找字典
    # get() 字典，函数返回指定键的值，如果值不在字典中返回默认值default。

In [16]:
def pat_to_dicts(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}#列表中.join

In [17]:
pattern = '?*P is very good and ?*X'
saying = "My dog is very good and my cat is very cute"

In [18]:
pat_match_with_seg_1 = pat_match_with_seg(pattern.split(), saying.split())

In [19]:
pat_match_with_seg_1

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

In [20]:
pat_match_with_seg_list = pat_to_dicts(pat_match_with_seg_1)

In [21]:
pat_match_with_seg_list

{'?P': 'My dog', '?X': 'my cat is very cute'}

In [29]:
subsitite('?P 和 ?X 真 好'.split() , pat_match_with_seg_list)

['My dog', '和', 'my cat is very cute', '真', '好']

In [31]:
" ".join(subsitite('?P 和 ?X 真 好'.split() , pat_match_with_seg_list))

'My dog 和 my cat is very cute 真 好'

In [ ]:
#例子

In [462]:
" ".join(subsitite("Why do you neeed ?X".split(), pat_to_dicts(pat_match_with_seg('I need ?*X'.split(), 
                                                                        "I need an iPhone".split()))))

'Why do you neeed an iPhone'

In [412]:
#示例

In [24]:
pattern = '?*P is very good and ?*X'
rule = '?P 和 ?X 真好'
saying = "My dog is very good and my cat is very cute"

In [25]:
def get_response(saying, rule):
    pat_match_with_seg_1  = pat_match_with_seg(pattern.split(),  saying.split())
    return" ".join(subsitite(rule.split(), pat_to_dicts(pat_match_with_seg_1)))

In [26]:
get_response(saying = saying ,rule = rule)

'My dog 和 my cat is very cute 真好'